Clean _Unmatched Dir

In [7]:
import os
import shutil
from pathlib import Path

OGE_Documents = Path("OGE_Documents/_Unmatched/")

moved_count, total_count = 0, 0

for file in OGE_Documents.glob("*.pdf"):
    total_count += 1
    if "278" in str(file):
        moved_count += 1
        #move this file to _Unmatched_cleaned
        #shutil.move(str(file), "OGE_Documents/_Unmatched_cleaned/" + file.name)

print(f"Moved {moved_count} files")
print(f"Total {total_count} files")

Moved 0 files
Total 163 files


Get the unmatched dir

In [1]:
from pathlib import Path

# Define the path to the _Unmatched directory
unmatched_dir = Path("OGE_Documents/_Unmatched_cleaned/")

# Get all PDF files
pdf_files = list(unmatched_dir.glob("*.pdf"))

# Or use rglob() for recursive search (in case there are subdirectories)
# pdf_files = list(unmatched_dir.rglob("*.pdf"))

print(f"Found {len(pdf_files)} PDF files")

#name <- pdf_file.name
#full path <- pdf_file

Found 12 PDF files


Simple matching

In [92]:
import re
from collections import defaultdict

mapping = defaultdict(list)
cleaned_names = []

#first clean the name
for pdf_file in pdf_files:
    name = pdf_file.name
    full_path = pdf_file

    cleaned_name = ""
    for char in name:
        if not char.isdigit():
            cleaned_name += char
        else:
            break

    cleaned_name = cleaned_name.split(" ")

    if len(cleaned_name) > 1:
        cleaned_name = cleaned_name[0] + " " + cleaned_name[1]

    else:
        cleaned_name = cleaned_name[0]

    cleaned_name = cleaned_name.replace("-", ", ")

    cleaned_names.append(cleaned_name)


print(cleaned_names)


['Tomney Brian', 'Keith, J, Krach, ', 'Steven, H, Fagin, ', 'Refus, C, Kemper, ', 'Vivek, H, Murth, ', 'Richard, M, Mills, Jr, ', 'Robert, .Sisson, ', 'Tod, M, Harper, ', 'Williams, Jr.,', 'Syed, Dilawar, ', 'R, Nicholas, Burns, ', 'Stone, Manning, Tracy', 'Thomas, R, Nides, ', 'Richard, Trumpka, Jr, ', 'Todd, M, Haepr, ', 'Steven, H, Fagin, ', 'Wilcox Gwynne', 'Smith, III,', 'Richardson, Sr.,', 'Todd, M, Haeper, ', 'Peter, Hoekstr, ', 'William, R, Evanina, ', 'Victoria, R, Kennedy, , ', 'Vashee, Dharmesh, ', 'William, R, Evanina, ', 'Whyche, Shaw, Oren', 'Rajkovich, Jr.,', 'Scott Kristi', 'Rebecca, J, Kelly, Slaughter, ', 'Thomas, A, Summers, , ', 'Steven, H, Fagin, ', 'Steven, H, Fagin, ', 'Robert, L, Sumwalt, III, ', 'Richard, Trumka, Jr, ', 'Robert, Silver, ', 'Van Vranken,', 'Rana, A, Gautam, ', 'Vidal, Katherine, ', 'Richard, Trumka, Jr, ', 'Trumka, Jr.', 'Keith, J, Krach, ', 'Robert, .Sisson, ', 'Puneet, Talawar, ', 'Scott, McGregor, ', 'Summer, Messinger, ', 'Richard, C, Parsch

In [ ]:
from thefuzz import fuzz
from thefuzz import process

mapping = {}

def fuzzy_match_name(cleaned_name, people_dict, threshold=50):
    """
    Perform fuzzy matching between a cleaned name and keys in peopleToPage dictionary.
    
    Args:
        cleaned_name: The cleaned name from the PDF file
        people_dict: The peopleToPage dictionary
        threshold: Minimum similarity score (0-100) to consider a match
        
    Returns:
        Tuple of (matched_key, page_number, score) or None if no good match found
    """
    # Get the best match using fuzzy matching
    best_match = process.extractOne(cleaned_name, people_dict.keys(), scorer=fuzz.ratio)
    
    if best_match and best_match[1] >= threshold:
        matched_key = best_match[0]
        page_number = people_dict[matched_key]
        score = best_match[1]
        return (matched_key, page_number, score)
    
    return None

# Perform fuzzy matching for all cleaned names
print(f"Starting fuzzy matching for {len(cleaned_names)} files...")
print(f"Total people in mapping: {len(peopleToPage)}")
print("-" * 80)

matched_count = 0
unmatched_count = 0

for i, cleaned_name in enumerate(cleaned_names):
    pdf_file = pdf_files[i]
    
    # Try fuzzy matching
    match_result = fuzzy_match_name(cleaned_name, cleaned_peopletopage, threshold=80)
    
    if match_result:
        matched_key, page_number, score = match_result
        # Add to mapping: key -> list of [file_path, page_number]
        mapping[matched_key].append([pdf_file, page_number])
        matched_count += 1
        print(f" Match ({score}%): '{cleaned_name}' → '{matched_key}' (Page {page_number})")
        print(f" File: {pdf_file.name}")
    else:
        unmatched_count += 1


print("-" * 80)
print(f"\n📊 Summary:")
print(f"  ✓ Matched: {matched_count}/{len(cleaned_names)}")
print(f"  ✗ Unmatched: {unmatched_count}/{len(cleaned_names)}")
print(f"  📁 Unique people matched: {len(mapping)}")


In [ ]:
import os
import shutil
from pathlib import Path

OGE_Documents = Path("OGE_Documents")

moved_count = 0
skipped_count = 0
missing_count = 0

for k, v in mapping.items():
    # v is a list of [file_path, page_number] pairs
    for item in v:
        file_path = item[0]  # The file to move
        page_number = item[1]  # The page number (e.g., 35)
        
        # Check if source file exists
        if not file_path.exists():
            print(f"Source missing: {file_path.name}")
            missing_count += 1
            continue
        
        # Construct the page directory path (e.g., OGE_Documents/Page_35)
        page_dir = OGE_Documents / f"Page_{page_number:02d}"
        
        # Construct the person/key directory inside the page (e.g., OGE_Documents/Page_35/Murthy, Vivek...)
        # Replace underscores with spaces in the key name
        k_cleaned = k.replace("_", " ")
        person_dir = page_dir / k_cleaned
        
        # Create the person directory if it doesn't exist
        if not person_dir.exists():
            person_dir.mkdir(parents=True, exist_ok=True)
        
        # Check if file already exists at destination
        dest_file = person_dir / file_path.name
        if dest_file.exists():
            print(f"Already at destination: {file_path.name}")
            skipped_count += 1
        else:
            # Move the file to the person directory
            shutil.move(str(file_path), str(person_dir))
            print(f"Moved: {file_path.name} -> {person_dir}")
            moved_count += 1

print(f"\n✓ Moved: {moved_count} files")
print(f"⊘ Skipped: {skipped_count} files (already at destination)")
print(f"⚠ Missing: {missing_count} files (source not found)")

In [86]:
# import shutil
# from pathlib import Path

# # Files to move back to _Unmatched_cleaned
# files_to_undo = [
#     "Steven-H-Fagin-2024-278.pdf",
#     "Steven-H-Fagin-2025-278.pdf",
#     "Steven-H-Fagin-10.27.2024-278T.pdf",
#     "Steven-H-Fagin-2023-278.pdf",
#     "R-Nicholas-Burns-03.22.2022-278T.pdf",
#     "Stone-Manning, Tracy  final278.pdf",
#     "Thomas-R-Nides-01.24.2022-278T.pdf",
#     "Wilcox Gwynne A.  final278.pdf",
#     "Richardson, Sr., Cardell K.  final278.pdf",
#     "William-R-Evanina-2021-278TERM.pdf",
#     "William-R-Evanina-11.19.2020-278T.pdf",
#     "Victoria-R-Kennedy--1.07.2025-278T.pdf",
#     "Vashee-Dharmesh-04.19.2023-278T.pdf",
#     "Whyche-Shaw, Oren  final278.pdf",
#     "Rebecca-J-Kelly-Slaughter-2020-278.pdf",
#     "Vidal-Katherine-2025-278TERM.pdf",
#     "Scott-McGregor-2020-278.pdf",
#     "Scott-McGregor-2021-278TERM.pdf",
#     "Richard-C-Parschall-05.22.2021-278T.pdf",
#     "Taylor, Michèle  final278.pdf",
#     "Thomas-Greenfield, Linda J.  final278.pdf",
#     "Syptak-Ramnath, Stephanie  final278.pdf",
#     "Slater-Chandler-Neysa-M-2024-278.pdf"
# ]

# # Destination folder
# unmatched_cleaned = Path("OGE_Documents/_Unmatched_cleaned")
# unmatched_cleaned.mkdir(exist_ok=True)

# # Search for these files in OGE_Documents
# oge_docs = Path("OGE_Documents")
# moved_back = 0
# not_found = 0

# print("Searching for files to move back...")
# print("-" * 80)

# for filename in files_to_undo:
#     # Search for the file in all subdirectories
#     found_files = list(oge_docs.rglob(filename))
    
#     if found_files:
#         for file_path in found_files:
#             # Skip if already in _Unmatched_cleaned
#             if "_Unmatched_cleaned" in str(file_path):
#                 print(f"⊘ Already in _Unmatched_cleaned: {filename}")
#                 continue
            
#             # Move back to _Unmatched_cleaned
#             dest = unmatched_cleaned / filename
#             try:
#                 shutil.move(str(file_path), str(dest))
#                 moved_back += 1
#                 print(f"✓ Moved back: {filename}")
#                 print(f"  From: {file_path}")
#             except Exception as e:
#                 print(f"✗ Error moving {filename}: {e}")
#     else:
#         not_found += 1
#         print(f"✗ Not found: {filename}")

# print("-" * 80)
# print(f"\n📊 Summary:")
# print(f"  ✓ Moved back: {moved_back}")
# print(f"  ✗ Not found: {not_found}")